## Convert wav to mp3

brew install ffmpeg to install ffmpeg before continuing

use python 3.7 - 3.10.10

In [2]:
from pydub import AudioSegment

AudioSegment.from_wav("../Data/testfile.wav").export("../Data/testfile.mp3", format="mp3")

<_io.BufferedRandom name='../Data/testfile.mp3'>

## Whisper AI on mp3

In [3]:
!pip install git+https://github.com/openai/whisper.git 


  Cloning https://github.com/openai/whisper.git to /private/var/folders/m0/7534pdbd5n16nwc4zl4gmb3h0000gn/T/pip-req-build-4f1oghbz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /private/var/folders/m0/7534pdbd5n16nwc4zl4gmb3h0000gn/T/pip-req-build-4f1oghbz
  Resolved https://github.com/openai/whisper.git to commit b5851c6c40e753606765ac45b85b298e3ae9e00d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.6 MB/s eta 0:00:00a 0:00:01
  Using cached more_itertools-9.1.0-py3-none-any.whl (54 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.0/701.0 kB 14.9 MB/s eta 0:00:0000:01
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 20.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28

In [4]:
import whisper
import os
import numpy as np
import torch

In [7]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
model = whisper.load_model("base", device=DEVICE)
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|███████████████████████████████████████| 139M/139M [00:10<00:00, 14.2MiB/s]


Model is multilingual and has 71,825,920 parameters.


In [9]:
audio = whisper.load_audio("../Data/testfile.mp3")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)

In [10]:
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

Detected language: en


In [12]:
options = whisper.DecodingOptions(language="en", without_timestamps=True, fp16 = False)
result = whisper.decode(model, mel, options)
print(result.text)

Hello, this is a test. We'll take your recording for myself. Then I will see if this can be transformed into text.


In [14]:
result = model.transcribe("../Data/testfile.mp3")
print(result["text"])

/Users/alexanderpoppe/miniforge3/envs/voice/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Hello, this is a test. We'll take your recording for myself. Then I will see if this can be transformed into text.


In [15]:
result = model.transcribe("../Data/testfile.wav")
print(result["text"])

/Users/alexanderpoppe/miniforge3/envs/voice/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Hello, this is a test. Well, take a recording for myself. Then I will see if this can be transformed into text.
